In [1]:
import phenopackets
import os
from os.path import isfile
from google.protobuf.json_format import MessageToDict, MessageToJson
from google.protobuf.json_format import Parse, ParseDict

In [3]:
from phenopackets import Phenopacket
import varcode as vc
#import myvariant as mv
import numpy as np
import pyensembl
import glob
import pandas as pd
from collections import defaultdict
import scipy.stats as stats

In [4]:
retinoblastoma = '../phenopackets/retinoblastoma.json'
nemalineMyopathy = '../phenopackets/nemalineMyopathy.json'
if not isfile(retinoblastoma):
    raise FileNotFoundError("Could not find phenopacket")

In [5]:
import json

with open(nemalineMyopathy) as f:
    data = f.read()
jsondata = json.loads(data)

NMphenopacket = Parse(json.dumps(jsondata), Phenopacket())

In [6]:
with open(retinoblastoma) as f:
    data = f.read()
jsondata = json.loads(data)

RBphenopacket = Parse(json.dumps(jsondata), Phenopacket())

ParseError: Failed to parse interpretations field: Failed to parse diagnosis field: Failed to parse genomicInterpretations field: Failed to parse variationDescriptor field: Failed to parse extensions field: Failed to parse value field: repeated field value must be in [] which is 40.0%.......

In [7]:
NMphenopacket.id

'arbitrary.id'

In [8]:
RBphenopacket

NameError: name 'RBphenopacket' is not defined

In [25]:
class Patient:
    def __init__(self, phenopackJson):
        if not isfile(phenopackJson):
            raise FileNotFoundError("Could not find phenopacket")
            
        with open(phenopackJson) as f:
            data = f.read()
        jsondata = json.loads(data)
        phenopack = Parse(json.dumps(jsondata), Phenopacket())
        
        self._id = phenopack.id
        self._phenopack = phenopack
        self._phenotype = self.__get_hpids()
        self._genotype = []
        self._disease = phenopack.diseases[0]
        if len(phenopack.interpretations) != 0:
            for i in range(len(phenopack.interpretations)):
                print(i)
                self._genotype.append(self.__get_variants(i))
        else:
            print('No interpretations found')
            self._genotype = None
                
        
    
    def __get_hpids(self):
        hp_ids = [x.type for x in self._phenopack.phenotypic_features if not x.excluded]
        return hp_ids
    
    def __get_variants(self, n):
        interp = self._phenopack.interpretations[n] 
        contig = []
        start = []
        ref = []
        alt = []

        for i in interp.diagnosis.genomic_interpretations:
            print(i)
            try:
                var_des = i.variant_interpretation.variation_descriptor.vcf_record
                contig.append(int(var_des.chrom))  ##.split('_')[1].split('.')[0])) #Create function to check chrom mapping
                ref.append(var_des.ref)
                alt.append(var_des.alt)
                start.append(var_des.pos)
            except:
                print("Error with Variant")
                print(i.variant_interpretation.variation_descriptor)
                continue
        
        myVars = []
        for i in range(len(contig)):
            myVar = vc.Variant(str(contig[i]), start[i], ref[i], alt[i], ensembl = pyensembl.ensembl_grch37)
            myVars.append(myVar)
        return myVars
    
    @property
    def disease(self):
        return self._disease
    
    
    @property
    def get_phenopacket(self):
        return self._phenopack
    
    @property
    def get_phenotypes(self):
        return self._phenotype
    
    @property
    def get_genotypes(self):
        geno = []
        if self._genotype == None:
            return None
        for i in self._genotype:
            for e in i:
                geno.append(e.short_description)
        return geno
    
    @property
    def get_var_effects(self):
        if self._genotype == None:
            return None
        effected = []
        for i in self._genotype:
            for e in i:
                effected.extend(e.effects().effects)
        return effected
    
    def is_missense(self):
        if self._genotype == None:
            return None
        miss = []
        for i in self.get_var_effects:
            if i.short_description.endswith("*") or not i.variant.is_snv:
                miss.append(False)
            else:
                miss.append(True)
        return miss
    
    def is_nonsense(self):
        if self._genotype == None:
            return None
        non = []
        for i in self.get_var_effects:
            if i.short_description.endswith("*") and i.variant.is_snv:
                non.append(True)
            else:
                non.append(False)
        return non
    
    def describe(self):
        stats = {
            "ID": self._phenopack.id,
            "Disease": self._phenopack.diseases,
            "Phenotypic Features": self.get_phenotypes,
            "Variants": self.get_genotypes,
            "Effects of Variants": self.get_var_effects,
            "Number Missense": sum(self.is_missense()),
            "Number Nonsense": sum(self.is_nonsense())
        }
        return stats

In [26]:
allPatients = defaultdict(Patient)
allDiseases = []
allDiseaseNames = set()

for file in glob.glob('../phenopackets/cohort/*.json'):
    fileName = os.path.basename(file)
    print(fileName)
    current = Patient(file)
    
    if current.get_genotypes is not None and len(current.get_genotypes) != 0:
        allPatients[fileName] = current
        for d in current.get_phenopacket.diseases:
            if d.term.id not in allDiseaseNames:
                allDiseases.append(d)
                allDiseaseNames.add(d.term.id)

for p in allPatients: print(allPatients[p].describe())
for p in allPatients: print(allPatients[p].get_phenopacket.diseases)


Naz_Villalba-2016-NLRP3-proband.json
0
subject_or_biosample_id: "proband"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "1"
      pos: 247587794
      ref: "C"
      alt: "T"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Abdul_Wahab-2016-GCDH-Patient_5.json
0
subject_or_biosample_id: "Patient 5"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "19"
      pos: 13007113
      ref: "G"
      alt: "A"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000136"
      label: "homozygous"
    }
  }
}

Papanastasiou-2010-STAT3-12_year_old_girl.json
0
subject_or_biosample_id: "12 year old girl"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_as

0
subject_or_biosample_id: "Patient 3"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "15"
      pos: 48776128
      ref: "C"
      alt: "T"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Mwasamwaja-2018-TGFB1-patient.json
0
subject_or_biosample_id: "patient"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "19"
      pos: 41848135
      ref: "G"
      alt: "A"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Asgharzade-2018-GIPC3-Ahv-14_23.json
0
subject_or_biosample_id: "Ahv-14:23"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "19"
      pos: 3586872


0
subject_or_biosample_id: "II-1"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "19"
      pos: 18897434
      ref: "GGCA"
      alt: "G"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Gonzalez-Perez-2009-CAV3-I1.json
0
subject_or_biosample_id: "I1"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "3"
      pos: 8775642
      ref: "G"
      alt: "A"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Noch-2017-ATP13A2-Case_1.json
0
subject_or_biosample_id: "Case 1"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "1"
      pos: 17322554
      ref: "G"
      

0
subject_or_biosample_id: "FKT118"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "9"
      pos: 34648167
      ref: "A"
      alt: "G"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

subject_or_biosample_id: "FKT118"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "9"
      pos: 34648167
      ref: "A"
      alt: "C"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Jagadisan-2017-PYGL-2-year_5-month_old_child.json
0
subject_or_biosample_id: "2-year 5-month old child"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "14"
      pos: 51382160
      ref: "C"

0
subject_or_biosample_id: "Patient 5"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "1"
      pos: 229567587
      ref: "T"
      alt: "A"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Jansen-2016-TMEM199-F3-II1.json
0
subject_or_biosample_id: "F3-II1"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "17"
      pos: 26684733
      ref: "G"
      alt: "C"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000136"
      label: "homozygous"
    }
  }
}

Li-2014-BBS4-4-year-old_female_patient.json
0
subject_or_biosample_id: "4-year-old female patient"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "15"
  

...

0
subject_or_biosample_id: "F6,II-2"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "10"
      pos: 134598646
      ref: "C"
      alt: "T"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000136"
      label: "homozygous"
    }
  }
}

Oud-2017-EXTL3-E_II-1.json
0
subject_or_biosample_id: "E:II-1"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "8"
      pos: 28575546
      ref: "A"
      alt: "G"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000136"
      label: "homozygous"
    }
  }
}

Van_Zwieten-2013-SLC4A1-c.1432-2A_T.json
0
subject_or_biosample_id: "c.1432-2A>T"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "17"
      pos: 42334914
      re

0
subject_or_biosample_id: "III-1"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "10"
      pos: 134599332
      ref: "T"
      alt: "A"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000136"
      label: "homozygous"
    }
  }
}

Gao-2019-RUNX2-Family_B_II1.json
0
subject_or_biosample_id: "Family_B_II1"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "6"
      pos: 45399750
      ref: "G"
      alt: "A"
    }
    molecule_context: genomic
    allelic_state {
      id: "GENO:0000135"
      label: "heterozygous"
    }
  }
}

Gowda-2017-MCOLN1-6_year_old_boy.json
0
subject_or_biosample_id: "6 year old boy"
interpretation_status: CAUSATIVE
variant_interpretation {
  variation_descriptor {
    vcf_record {
      genome_assembly: "GRCh37"
      chrom: "19"
      pos: 75928

{'ID': 'PMID:28984260-Jagadisan-2017-PYGL-2-year_5-month_old_child', 'Disease': [term {
  id: "OMIM:232700"
  label: "GLYCOGEN STORAGE DISEASE VI; GSD6"
}
], 'Phenotypic Features': [id: "HP:0003270"
label: "Abdominal distention"
, id: "HP:0004322"
label: "Short stature"
, id: "HP:0002910"
label: "Elevated hepatic transaminase"
, id: "HP:0003124"
label: "Hypercholesterolemia"
, id: "HP:0002155"
label: "Hypertriglyceridemia"
, id: "HP:0001531"
label: "Failure to thrive in infancy"
, id: "HP:0001405"
label: "Periportal fibrosis"
, id: "HP:0002240"
label: "Hepatomegaly"
], 'Variants': ['chr14 g.51382160C>A'], 'Effects of Variants': [PrematureStop(variant=Variant(contig='14', start=51382160, ref='C', alt='A', reference_name='GRCh37'), transcript_name=PYGL-001, transcript_id=ENST00000216392, effect_description=p.E433*), NoncodingTranscript(variant=Variant(contig='14', start=51382160, ref='C', alt='A', reference_name='GRCh37'), transcript_name=PYGL-011, transcript_id=ENST00000528757), Prematu

{'ID': 'PMID:27732904-Zhang-2016-RPS19-patient', 'Disease': [term {
  id: "OMIM:105650"
  label: "DIAMOND-BLACKFAN ANEMIA 1; DBA1"
}
], 'Phenotypic Features': [id: "HP:0001972"
label: "Macrocytic anemia"
], 'Variants': ['chr19 g.42373179_42373179delG'], 'Effects of Variants': [FrameShift(variant=Variant(contig='19', start=42373179, ref='G', alt='', reference_name='GRCh37'), transcript_name=RPS19-005, transcript_id=ENST00000221975, effect_description=p.R10fs), FrameShift(variant=Variant(contig='19', start=42373179, ref='G', alt='', reference_name='GRCh37'), transcript_name=RPS19-006, transcript_id=ENST00000593863, effect_description=p.R84fs), IncompleteTranscript(variant=Variant(contig='19', start=42373179, ref='G', alt='', reference_name='GRCh37'), transcript_name=RPS19-007, transcript_id=ENST00000598261), NoncodingTranscript(variant=Variant(contig='19', start=42373179, ref='G', alt='', reference_name='GRCh37'), transcript_name=RPS19-008, transcript_id=ENST00000598399), FrameShift(vari

{'ID': 'PMID:30681580-Yao-2019-FGFR3-VI-5', 'Disease': [term {
  id: "OMIM:146000"
  label: "HYPOCHONDROPLASIA; HCH"
}
], 'Phenotypic Features': [id: "HP:0002970"
label: "Genu varum"
, id: "HP:0100864"
label: "Short femoral neck"
, id: "HP:0008450"
label: "Narrow vertebral interpedicular distance"
, id: "HP:0001156"
label: "Brachydactyly"
, id: "HP:0100866"
label: "Short iliac bones"
], 'Variants': ['chr4 g.1805540C>T'], 'Effects of Variants': [Substitution(variant=Variant(contig='4', start=1805540, ref='C', alt='T', reference_name='GRCh37'), transcript_name=FGFR3-001, transcript_id=ENST00000260795, effect_description=p.S351F), Intronic(variant=Variant(contig='4', start=1805540, ref='C', alt='T', reference_name='GRCh37'), transcript_name=FGFR3-201, transcript_id=ENST00000340107), Intronic(variant=Variant(contig='4', start=1805540, ref='C', alt='T', reference_name='GRCh37'), transcript_name=FGFR3-002, transcript_id=ENST00000352904), Intronic(variant=Variant(contig='4', start=1805540, re

In [12]:
print(f"Number of patients {len(allPatients)}")

Number of patients 373


In [14]:
class Counts:
    def __init__(self, disease):
        self._disease = disease.term.id
        self._diseaseName = disease.term.label
        self._totals = pd.DataFrame({'key':["Missense","Nonsense"]}).set_index('key')
    
    
    def add_patient(self, patient):
        if not isinstance(patient, Patient):
            raise NotAPatientError("ERROR: Input must be of class Patient")
        hasDisease = False
        for d in patient.describe()["Disease"]:
            if d.term.id.upper() == self._disease:
                hasDisease = True
        if not hasDisease:
            #print("Patient " + patient._id +" does not have disease " + self._disease)
            return 
        
        miss = sum(patient.is_missense())
        nons = sum(patient.is_nonsense())
        
        if miss == 0 and nons == 0:
            #print("Patient " + patient._id + " has no missense or nonsense mutations")
            return 
        for p in patient.get_phenotypes:
            if p.id not in self._totals.columns:
                df1 = pd.DataFrame({'key':["Missense", "Nonsense"],
                    p.id: [miss, nons]})
                df1 = df1.set_index('key')
                self._totals = self._totals.join(df1, how='outer')
            else:
                self._totals.at["Missense", p.id] += miss
                self._totals.at["Nonsense", p.id] += nons
        return self._totals
    
    def run_chi2(self):
        """
        self._totals is a 2x2 array with [0,0] = category 1, has HPO, 
        [0,1] category 1 does not have hpo
        
        """
        if len(self._totals.columns) != 0:
            if self._totals.sum(axis=1).at['Nonsense'] !=0 and self._totals.sum(axis=1).at['Missense'] != 0:
                chi2, pval, dof, exp = stats.chi2_contingency(self._totals)
            else:
                #print("There are either no Nonsense or no Missense Mutations")
                return
        else:
            #print("Add patients to run a Chi2 Test")
            return
        print("The p-value is: " + str(pval) )
        if pval <= 0.05:
            print("We can conclude the Missense vs Nonsense mutations are NOT independent of the Phenotypes for disease " + self._disease)
            return 'False'
        else:
            print("We can conclude the Missense vs Nonsense mutations are independent of the Phenotypes for disease " + self._disease)
            return 'True'



In [50]:
allDiseaseCounts = defaultdict(Counts)
allIndependent = defaultdict(Counts)
allNotIndependent = defaultdict(Counts)
import scipy


def has_hpo(pat, hpo):
    for h in pat.get_phenotypes:
        if h.id == hpo:
            return True
    return False

for d in allDiseases:
    #print(f"d: {d}")
    patients_with_d = [p for p in allPatients.values() if p.disease.term.id == d.term.id]
    n_dis = len(patients_with_d)
    if n_dis < 15: continue
    print(f"Patients with disease {d}: n={n_dis}")
    # fisher exact test is what we actually need
    all_hpo = set()
    total_pat = len(patients_with_d)
    for pat in patients_with_d:
        for hpo in pat.get_phenotypes:
            all_hpo.add(hpo.id)
    print(f"Total hpo terms: {len(all_hpo)}")
    for hpo_id in all_hpo:
        miss_with_hpo = len([ pat for pat in patients_with_d if has_hpo(pat, hpo_id) and pat.is_missense()])
        miss_without_hpo = len([ pat for pat in  patients_with_d if not has_hpo(pat, hpo_id) and pat.is_missense()])
        nons_with_hpo = len([ pat  for pat in patients_with_d if has_hpo(pat, hpo_id) and pat.is_nonsense()])
        nons_without_hpo = len([ pat for pat in patients_with_d if not has_hpo(pat, hpo_id) and pat.is_nonsense()])
        print(f"HPO {hpo_id}: miss_with_hpo {miss_with_hpo} miss_without_hpo {miss_without_hpo} nons_with_hpo{nons_with_hpo} nons_without_hpo{nons_without_hpo}")
        table = np.array([[miss_with_hpo, miss_without_hpo], [nons_with_hpo, nons_without_hpo]])
        oddsr, p =  scipy.stats.fisher_exact(table, alternative='two-sided')
        print(f"p: {p}")
        

Patients with disease term {
  id: "OMIM:616900"
  label: "Hypotonia, infantile, with psychomotor retardation and characteristic facies 3"
}
: n=19
Total hpo terms: 130
HPO HP:0000717: miss_with_hpo 1 miss_without_hpo 18 nons_with_hpo1 nons_without_hpo18
p: 1.0
HPO HP:0001263: miss_with_hpo 4 miss_without_hpo 15 nons_with_hpo4 nons_without_hpo15
p: 1.0
HPO HP:0000337: miss_with_hpo 1 miss_without_hpo 18 nons_with_hpo1 nons_without_hpo18
p: 1.0
HPO HP:0002093: miss_with_hpo 4 miss_without_hpo 15 nons_with_hpo4 nons_without_hpo15
p: 1.0
HPO HP:0006191: miss_with_hpo 2 miss_without_hpo 17 nons_with_hpo2 nons_without_hpo17
p: 1.0
HPO HP:0001249: miss_with_hpo 8 miss_without_hpo 11 nons_with_hpo8 nons_without_hpo11
p: 1.0
HPO HP:0002079: miss_with_hpo 6 miss_without_hpo 13 nons_with_hpo6 nons_without_hpo13
p: 1.0
HPO HP:0000414: miss_with_hpo 3 miss_without_hpo 16 nons_with_hpo3 nons_without_hpo16
p: 1.0
HPO HP:0012020: miss_with_hpo 1 miss_without_hpo 18 nons_with_hpo1 nons_without_hpo18
p

HPO HP:0006543: miss_with_hpo 1 miss_without_hpo 18 nons_with_hpo1 nons_without_hpo18
p: 1.0
HPO HP:0007359: miss_with_hpo 3 miss_without_hpo 16 nons_with_hpo3 nons_without_hpo16
p: 1.0
HPO HP:0000340: miss_with_hpo 4 miss_without_hpo 15 nons_with_hpo4 nons_without_hpo15
p: 1.0
HPO HP:0012736: miss_with_hpo 5 miss_without_hpo 14 nons_with_hpo5 nons_without_hpo14
p: 1.0
HPO HP:0000490: miss_with_hpo 5 miss_without_hpo 14 nons_with_hpo5 nons_without_hpo14
p: 1.0
HPO HP:0002902: miss_with_hpo 1 miss_without_hpo 18 nons_with_hpo1 nons_without_hpo18
p: 1.0
HPO HP:0000565: miss_with_hpo 2 miss_without_hpo 17 nons_with_hpo2 nons_without_hpo17
p: 1.0
HPO HP:0000486: miss_with_hpo 3 miss_without_hpo 16 nons_with_hpo3 nons_without_hpo16
p: 1.0
HPO HP:0012032: miss_with_hpo 1 miss_without_hpo 18 nons_with_hpo1 nons_without_hpo18
p: 1.0
HPO HP:0001239: miss_with_hpo 1 miss_without_hpo 18 nons_with_hpo1 nons_without_hpo18
p: 1.0
HPO HP:0000280: miss_with_hpo 5 miss_without_hpo 14 nons_with_hpo5 non

HPO HP:0008619: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0000218: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0005824: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0012168: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0010864: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0002194: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0002791: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0000954: miss_with_hpo 2 miss_without_hpo 16 nons_with_hpo2 nons_without_hpo16
p: 1.0
HPO HP:0000540: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0000957: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0010704: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 non

HPO HP:0000358: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0000396: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0002247: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0000567: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0002360: miss_with_hpo 2 miss_without_hpo 16 nons_with_hpo2 nons_without_hpo16
p: 1.0
HPO HP:0004755: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0001513: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0030842: miss_with_hpo 1 miss_without_hpo 17 nons_with_hpo1 nons_without_hpo17
p: 1.0
HPO HP:0000961: miss_with_hpo 2 miss_without_hpo 16 nons_with_hpo2 nons_without_hpo16
p: 1.0
HPO HP:0001629: miss_with_hpo 5 miss_without_hpo 13 nons_with_hpo5 nons_without_hpo13
p: 1.0
HPO HP:0012448: miss_with_hpo 2 miss_without_hpo 16 nons_with_hpo2 non

In [23]:
allDiseaseCounts = defaultdict(Counts)
allIndependent = defaultdict(Counts)
allNotIndependent = defaultdict(Counts)

for d in allDiseases:
    patients_with_d = [p for p in allPatients if p.disease.term.id == d]
    
    allDiseaseCounts[d.term.id] = Counts(d)
    for p in allPatients:
        allDiseaseCounts[d.term.id].add_patient(allPatients[p])
    fin = allDiseaseCounts[d.term.id].run_chi2()
    if fin == 'True':
        allIndependent[d.term.id] = allDiseaseCounts[d.term.id]
    elif fin == 'False':
        allNotIndependent[d.term.id] = allDiseaseCounts[d.term.id]



AttributeError: 'str' object has no attribute 'disease'

In [304]:
for i in allNotIndependent:
    print(allNotIndependent[i]._diseaseName)


Hypotonia, infantile, with psychomotor retardation and characteristic facies 3
CHARGE SYNDROME
Spastic paraplegia 76, autosomal recessive
Parkinson disease 23, autosomal recessive, early onset
Neurodevelopmental disorder with or without anomalies of the brain, eye, or heart
COCKAYNE SYNDROME A; CSA
TUBEROUS SCLEROSIS 1; TSC1
